# 예보 데이터 불러오기
[한국동서발전 태양광 발전량 현황 인증키 확인](https://data.go.kr/tcs/dss/selectFileDataDetailView.do?publicDataPk=15003553#tab-layer-openapi)

[인증키 주소(로그인필요..)](https://data.go.kr/iim/api/selectAPIAcountView.do)

In [ ]:
! pip install xmltodict

In [2]:
import pandas as pd
from urllib.parse import urlencode, quote_plus
from urllib.request import urlopen
import json
import xmltodict

In [1]:
# API
ServiceKey = input('encoded api key를 입력해주세요 > ')

In [61]:
# JSON으로 파일받은 뒤 정리하기
url_base = 'https://api.odcloud.kr/api/15003553/v1/uddi:34a871be-f9c5-497a-9c69-a9770ff0ec8d'
key = ServiceKey
queryParams = urlencode({
    quote_plus('page'): 1,      # 불러오고 싶은만큼 가져오기
    quote_plus('perPage'): 10,
})

url = f'{url_base}?serviceKey={key}&{queryParams}'
response = urlopen(url)
results = response.read().decode('utf-8')

data = pd.read_json(results)

data_energy = pd.DataFrame()
for i in range(len(data)):
    dict_data = data.loc[i, 'data']
    time = dict_data.pop('시간')
    region = dict_data.pop('태양광명')
    
    time_data = pd.DataFrame([b]).T
    time_data.reset_index(inplace=True)
    time_data.columns = ['hours', '발전량']
    
    time_data['시간'] = time
    time_data['date'] = time_data['시간'] + ' ' + time_data['hours'] + ':00:00'
    time_data['hours'] = time_data['hours'].astype('int64')
    time_data.sort_values('hours')
    
    col_return = ['date', '발전량']
    
#     print(time_data[col_return])
    data_energy = pd.concat([data_energy, time_data[col_return]])

data_energy

,date,발전량
0,2018-01-01 1:00:00,0
1,2018-01-01 10:00:00,216
2,2018-01-01 11:00:00,349
3,2018-01-01 12:00:00,344
4,2018-01-01 13:00:00,374
...,...,...
19,2018-01-10 5:00:00,0
20,2018-01-10 6:00:00,0
21,2018-01-10 7:00:00,0
22,2018-01-10 8:00:00,0


# 여기 밑으로는 무시

In [4]:

url_base = 'https://api.odcloud.kr/api/15003553/v1/uddi:34a871be-f9c5-497a-9c69-a9770ff0ec8d'
key = ServiceKey
queryParams = urlencode({
    quote_plus('page'): 1,      # 불러오고 싶은만큼 가져오기
    quote_plus('perPage'): 10,
    quote_plus('returnType'): 'XML', # 예보 발표 날짜
})

url = f'{url_base}?serviceKey={key}&{queryParams}'
response = urlopen(url)
results = response.read().decode('utf-8')
results_json = xmltodict.parse(results)

data = json.loads(json.dumps(results_json))

# if 'ERROR' in data['response']['header']['resultMsg']:
#     print('Fail.')
# else:
#     print('Success!')

In [12]:

url_base = 'https://api.odcloud.kr/api/15003553/v1/uddi:34a871be-f9c5-497a-9c69-a9770ff0ec8d'
key = ServiceKey
queryParams = urlencode({
    quote_plus('page'): 1,      # 불러오고 싶은만큼 가져오기
    quote_plus('perPage'): 10,
#     quote_plus('returnType'): 'XML', # 예보 발표 날짜
})

url = f'{url_base}?serviceKey={key}&{queryParams}'
response = urlopen(url)
results = response.read().decode('utf-8')
# results_json = xmltodict.parse(results)

# data = json.loads(json.dumps(results_json))

# if 'ERROR' in data['response']['header']['resultMsg']:
#     print('Fail.')
# else:
#     print('Success!')

In [45]:
# 폐기하기

a = pd.read_json(results)
b = a.loc[0, 'data']
# pd.DataFrame([b])
time = b.pop('시간')
region = b.pop('태양광명')
c = pd.DataFrame([b]).T
d = pd.DataFrame()
c.reset_index(inplace=True)
c.columns = ['hours', '발전량']
c['hours'] = c['hours'].apply(int)
c['hours'] = c['hours'].apply(lambda x: f'{x-1:0>2d}')
c['시간'] = time + ' '
c['date'] = c['시간'] + c['hours']
c['date'] = pd.to_datetime(c['date'])

In [54]:
a = pd.read_json(results)
b = a.loc[0, 'data']
# pd.DataFrame([b])
time = b.pop('시간')
region = b.pop('태양광명')
c = pd.DataFrame([b]).T

c.reset_index(inplace=True)
c.columns = ['hours', '발전량']
c['시간'] = time
c['date'] = c['시간'] + ' ' + c['hours'] + ':00:00'
c['hours'] = c['hours'].astype('int64')
c.sort_values('hours')

,hours,발전량,시간,date
0,1,0,2018-01-01,2018-01-01 1:00:00
11,2,0,2018-01-01,2018-01-01 2:00:00
17,3,0,2018-01-01,2018-01-01 3:00:00
18,4,0,2018-01-01,2018-01-01 4:00:00
19,5,0,2018-01-01,2018-01-01 5:00:00
20,6,0,2018-01-01,2018-01-01 6:00:00
21,7,0,2018-01-01,2018-01-01 7:00:00
22,8,0,2018-01-01,2018-01-01 8:00:00
23,9,45,2018-01-01,2018-01-01 9:00:00
1,10,216,2018-01-01,2018-01-01 10:00:00


In [53]:
c

,hours,발전량,시간,date
0,1,0,2018-01-01,2018-01-01 1:00:00
1,10,216,2018-01-01,2018-01-01 10:00:00
2,11,349,2018-01-01,2018-01-01 11:00:00
3,12,344,2018-01-01,2018-01-01 12:00:00
4,13,374,2018-01-01,2018-01-01 13:00:00
5,14,407,2018-01-01,2018-01-01 14:00:00
6,15,486,2018-01-01,2018-01-01 15:00:00
7,16,281,2018-01-01,2018-01-01 16:00:00
8,17,50,2018-01-01,2018-01-01 17:00:00
9,18,0,2018-01-01,2018-01-01 18:00:00


In [5]:
# data 대강 확인하기
data

{'results': {'currentCount': '10',
  'data': {'item': [{'col': [{'@name': '8', '#text': '0'},
      {'@name': '태양광명', '#text': '당진태양광'},
      {'@name': '1', '#text': '0'},
      {'@name': '10', '#text': '216'},
      {'@name': '16', '#text': '281'},
      {'@name': '17', '#text': '50'},
      {'@name': '20', '#text': '0'},
      {'@name': '21', '#text': '0'},
      {'@name': '12', '#text': '344'},
      {'@name': '15', '#text': '486'},
      {'@name': '19', '#text': '0'},
      {'@name': '22', '#text': '0'},
      {'@name': '23', '#text': '0'},
      {'@name': '3', '#text': '0'},
      {'@name': '13', '#text': '374'},
      {'@name': '14', '#text': '407'},
      {'@name': '18', '#text': '0'},
      {'@name': '2', '#text': '0'},
      {'@name': '9', '#text': '45'},
      {'@name': '시간', '#text': '2018-01-01'},
      {'@name': '11', '#text': '349'},
      {'@name': '24', '#text': '0'},
      {'@name': '4', '#text': '0'},
      {'@name': '5', '#text': '0'},
      {'@name': '6', '#text': 

In [11]:
a = data['results']['data']['item'][0]
pd.DataFrame(a['col'])


,@name,#text
0,8,0
1,태양광명,당진태양광
2,1,0
3,10,216
4,16,281
5,17,50
6,20,0
7,21,0
8,12,344
9,15,486


In [ ]:
logs = data['response']['body']['items']['item']
df = pd.DataFrame()

for log in logs:
    s_tmp = pd.Series(log)
    df_tmp = pd.DataFrame(s_tmp)
    df = pd.concat([df, df_tmp.T])

def fcst_time(time):
    temp = f'{time[:4]}-{time[4:6]}-{time[6:8]} {time[9:11]}:{time[11:]}'
    datetime = pd.to_datetime(temp)
    return datetime

df['time'] = df[['fcstDate', 'fcstTime']].apply(lambda t: ' '.join(t), axis=1)
df['time'] = df['time'].apply(fcst_time)

fcst_df = pd.DataFrame(columns=['time', 'fcst_temp', 'fcst_hum', 'fcst_wspeed',
                                'fcst_wdirect', 'fcst_cloud'])

df